# Interval Bridge Validation Tool

### Table of Contents
* [Front Matter](#frontmatter)
* [Modifiable Parameters](#parameters)
* [Imports and Setup Parameters](#imports)
* [Query the database using the Workflow Manager Service](#queryWM)
* [Query the database directly](#queryDB)
* [Compare the returns](#compare)
    * [Pull the Workflow Manager results](#pullWM)
    * [Pull the database results](#pullDB)
    * [Convert database interval records to 5min intervals](#convert)
    * [Print out results and mark mismatches](#printout)

## Validating Auto Network Processing Stage<a class="anchor" id="frontmatter"></a>

<u>**Background**</u>: 
* The Workflow Manager Service provides HTTP endpoints used to query 
Intevals. Workflow Manager Service uses the Inteval bridge component 
to query intervals from a legacy Oracle database, translates that data
from the legacy CSS data model to the GMS processing interval
COI, and returns JSON formatted results to the user.

<u>**What this tool does**</u>: 
* Validates that data acquired via the Workflow Manager Service
matches the expected results from the database tables. This notebook validates the Auto Network processing stage.  The AL1, Auto Post-AL1, and AL2 processing stages are validated in a separate notebook.

<u>**Currently the Workflow Manager has one OSD endpoint available 
for retrieving intervals:**</u>:
* Retrieve by intervals by ID and time range (workflow-manager/interval/stage/query/ids-timerange)

<u>**Objects available for retrieval**</u>:
* **Intervals**:
   * Using a list of stage IDs/names, a start time, and an end time, 
     returns the intervals and associated information 
   * Information includes:
     - processing stages (Auto Network, AL1, Auto Post-AL1, AL2)
     - processing sequences and activities (auto sequences, Event Review, Scan)
     - active analysts
     - interval status (Skipped, Not Started, In Progress, Not Complete, Complete, Failed)
     - processing start time
     - processing end time
     - storage time
     - modification time
     - percent available

## Modifiable Parameters<a class="anchor" id="parameters"></a>

In [1]:
################################################################
# START AND END TIMES FOR QUERY
################################################################
startTime_datetime = "2019-01-01T00:00:00Z"
endTime_datetime = "2019-02-01T00:00:00Z"

################################################################
# WORKFLOW MANAGER SERVICE URL
################################################################
deployment = 'deployment'
base_url = 'https://' + deployment + 'base_url/'
url_ext = 'workflow-manager-service/workflow-manager/' \
             'interval/stage/query/ids-timerange'
service_url = base_url + url_ext

################################################################
# DATABASE PARAMETERS
################################################################
### create_engine params for sqlalchemy
### format is dialect[+driver]://user:passwrd@host/service name
dbinfo = 'dialect[+driver]://user:passwrd@host/service name'

# Declare schema, database name
schema ='schema'
db = 'database'

## Imports and Setup Parameters<a class="anchor" id="imports"></a>

In [2]:
# Note, need cx_oracle installed for sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.sql import text
import pisces.schema.css3 as css
import cx_Oracle
import json 
import requests
import time
from datetime import datetime, time, timedelta
import numpy as np

# Manually declare headers for requests to endpoint 
headers = {'Accept': 'application/json',
       'Content-Type': 'application/json'}

# Convert start and end times to epoch times
startTime_epochTime = (datetime.strptime(startTime_datetime, "%Y-%m-%dT%H:%M:%SZ") - datetime.utcfromtimestamp(0)).total_seconds()
endTime_epochTime = (datetime.strptime(endTime_datetime, "%Y-%m-%dT%H:%M:%SZ") - datetime.utcfromtimestamp(0)).total_seconds()

## Query the database using the Workflow Manager Service<a class="anchor" id="queryWM"></a>

In [3]:
# Build JSON of input information
#     Input is a list of stage IDs/names, a start time, and an end time
#     startTime and endTime in datetime format, not epoch time
body = {
    "startTime": startTime_datetime,
    "endTime": endTime_datetime,
    "stageIds": [{"name": "Auto Network"}]
}

# Make a request to the service url
ret = requests.post(service_url, json=body, headers=headers)
wm_resp = ret.json()

## Query the database directly<a class="anchor" id="queryDB"></a>

In [4]:
#  Create engine to connect to the database
e = create_engine(dbinfo)

# Make a base that targets the right schema, where schema 
#    equals your database name
GMS = declarative_base(metadata=MetaData(schema=schema))

# Using that GMS base, create classes for the appropriate tables so that metadata fields 
# will be available to utilize in queries. Using the regular CSS schema because these are
# just metadata fields. 
class Affiliation(GMS, css.Affiliation):
    __tablename__ = 'INTERVAL'

# Connect to the database
connection = e.connect()

# Create query for the database
query = text("SELECT * from GMS_GLOBAL.INTERVAL \
             where (class='NET') \
             AND time >= '{}' \
             AND endtime <= '{}' \
             AND NOT name='FAL' \
             order by time".format(startTime_epochTime, endTime_epochTime))

# Query the database
result = connection.execute(query)

# Convert sql object to array for easy reading
rows = result.fetchall()

## Compare the returns from querying with the Workflow Manager and querying the database directly<a class="anchor" id="compare"></a>

### Pull the start times, stage names, and statuses from the Workflow Manager results<a class="anchor" id="pullWM"></a>

In [5]:
wm_startTime=[]
wm_endTime=[]
wm_modificationTime=[]
wm_stage=[]
wm_status=[]
for i in wm_resp.keys():
    for j in range(len(wm_resp[i])):
        wm_startTime.append(wm_resp[i][j]["startTime"])
        wm_endTime.append(wm_resp[i][j]["endTime"])
        wm_modificationTime.append(wm_resp[i][j]["modificationTime"])
        wm_stage.append(wm_resp[i][j]["name"])
        wm_status.append(wm_resp[i][j]["status"])

# Sort by start times, then stage name
wm_startTime, wm_stage, wm_status, wm_endTime, wm_modificationTime = \
    zip(*sorted(zip(wm_startTime, wm_stage, wm_status, wm_endTime, wm_modificationTime)))

### Pull the start times and stage names from the database results<a class="anchor" id="pullDB"></a>

In [6]:
# Pull out info from database return
db_class = []
db_name = []
db_startTime = []
db_endTime = []
db_modificationTime = []
db_status = []
db_analysts=[]
for i in rows:
    db_class.append(i[1])
    db_name.append(i[2])
    db_startTime.append(i[3])
    db_endTime.append(i[4])
    db_status.append(i[5])
    db_analysts.append(i[6])
    db_modificationTime.append(i[10])

# Convert database info to Workflow Manager terminology
db_startDateTime = []
db_endDateTime = []
db_stage = ['UNKNOWN'] * np.size(db_status)
db_statusWM = ['UNKNOWN'] * np.size(db_status)
for i in range(len(db_startTime)):
    
    # Convert to datetime from epoch time
    db_startDateTime.append(datetime.utcfromtimestamp(db_startTime[i]).strftime("%Y-%m-%dT%H:%M:%SZ"))
    db_endDateTime.append(datetime.utcfromtimestamp(db_endTime[i]).strftime("%Y-%m-%dT%H:%M:%SZ"))

    # Convert to Workflow Manager stage names
    if db_class[i] == "NET":
        db_stage[i] = 'Auto Network'
    else:
        db_stage[i] = 'UNKNOWN'

    # Convert to Workflow Manager statuses
    if db_status[i] in ['late-done', 'done', 'network-done']:
        db_statusWM[i] = "COMPLETE"
    elif db_status[i] in ['pending', 'queued', 'skipped']:
        db_statusWM[i] = "NOT_STARTED"
    elif db_status[i].startswith('failed'):
        db_statusWM[i] = "FAILED"
    elif db_status[i].endswith('start'):
        db_statusWM[i] = "IN_PROGRESS"
    else:
        db_statusWM[i]="UNKNOWN"
        
# Sort by start times, then stage names
db_startDateTime, db_stage, db_status, db_statusWM, db_endDateTime, db_modificationTime = zip(*sorted(zip(db_startDateTime, db_stage, db_status, db_statusWM, db_endDateTime, db_modificationTime)))

### Convert database interval records to 5min intervals<a class="anchor" id="convert"></a>

In [7]:
# Declare interval length (minutes)
intLen = 5  

# Instantiate converted variables
jnb_startDateTime = []
jnb_stage = []
jnb_status =[]
jnb_statusWM = []
jnb_endDateTime = []
jnb_modificationTime = []

# Instantiate array of start times for intervals
times = range(np.int(startTime_epochTime),np.int(endTime_epochTime), intLen * 60)

for i in times:
    
    # Find database intervals that encompass time in this 5 minute interval (startTime before, endTime after)
    ind = np.where(np.logical_and(np.array(db_startTime) <= i, np.array(db_endTime) >= i + intLen * 60))[0]
    
    # If intervals exists in this time
    if ind.size != 0:
        
        # Find the interval with the latest modification time
        ind2=np.argmax(np.array(db_modificationTime)[ind])

        rec = ind[ind2]
    
        # Assign intervals stats from inteval with latest modification time
        jnb_startDateTime.append(datetime.utcfromtimestamp(i).strftime("%Y-%m-%dT%H:%M:%SZ"))
        jnb_stage.append(db_stage[rec])
        jnb_status.append(db_status[rec])
        jnb_statusWM.append(db_statusWM[rec])
        jnb_endDateTime.append(datetime.utcfromtimestamp(i + intLen * 60).strftime("%Y-%m-%dT%H:%M:%SZ"))
        jnb_modificationTime.append(np.array(db_modificationTime)[rec])

### Print out results and mark mismatches<a class="anchor" id="printout"></a>

In [8]:
# Count differences between workflow manager and database results
diffct=0
for i in range(len(wm_startTime)):
    if(wm_startTime[i] != jnb_startDateTime[i] or wm_stage[i] != jnb_stage[i] or wm_status[i] != jnb_statusWM[i]):
        diffct+=1
        
print("Workflow Manager\t\t\t\tDatabase\t\t\t\tMismatch?")
print("Number of records: {}\t\t\t\tNumber of records: {}\t\t\tTotal = {}".format(len(wm_status), len(jnb_status), diffct))
print("-------------------------------------\t\t-------------------------------------\t---------")
print("Start Time\t\tStage/Status\t\tStart Time\t\tStatus")
for i in range(len(wm_status)):
    if wm_startTime[i] != jnb_startDateTime[i] or wm_stage[i] != jnb_stage[i] or wm_status[i] != jnb_statusWM[i]:
        print("\n{:<24}{:<24}{:<24}{:<16}Mismatch".format(wm_startTime[i],wm_stage[i],jnb_startDateTime[i],jnb_stage[i]))
        print("{:<24}{:<32}{:<16}{:<1}".format("", wm_status[i], "", jnb_statusWM[i]))
    else:
        print("\n{:<24}{:<24}{:<24}{:<16}".format(wm_startTime[i],wm_stage[i],jnb_startDateTime[i],jnb_stage[i]))
        print("{:<24}{:<32}{:<16}{:<1}".format("", wm_status[i], "", jnb_statusWM[i]))

Workflow Manager				Database				Mismatch?
Number of records: 308				Number of records: 308			Total = 0
-------------------------------------		-------------------------------------	---------
Start Time		Stage/Status		Start Time		Status

2019-01-04T23:00:00Z    Auto Network            2019-01-04T23:00:00Z    Auto Network    
                        COMPLETE                                        COMPLETE

2019-01-04T23:05:00Z    Auto Network            2019-01-04T23:05:00Z    Auto Network    
                        COMPLETE                                        COMPLETE

2019-01-04T23:10:00Z    Auto Network            2019-01-04T23:10:00Z    Auto Network    
                        COMPLETE                                        COMPLETE

2019-01-04T23:15:00Z    Auto Network            2019-01-04T23:15:00Z    Auto Network    
                        COMPLETE                                        COMPLETE

2019-01-04T23:20:00Z    Auto Network            2019-01-04T23:20:00Z    Auto Net